In [1]:
#import dependencies
import requests
import pandas as pd
import numpy as np
import os

In [2]:
script_dir = os.path.dirname("D:\CODING_BOOTCAMP\project_3\project_three\project_three.ipynb")

In [3]:
#Make API call
#URL of API
url = "https://data.montgomerycountymd.gov/api/views/v76h-r7br/rows.json?accessType=DOWNLOAD"
# Fetch data from the API
response = requests.get(url)
data = response.json()

In [4]:
# Extract the rows data from the JSON response
rows = data.get("data", [])

# Extract column names from the JSON response
columns = [col["fieldName"] for col in data.get("meta", {}).get("view", {}).get("columns", [])]

# Convert the data to a Pandas DataFrame
pandas_df = pd.DataFrame(rows, columns=columns)


In [5]:
#remove meta columns
columns_to_drop = [':sid', ':id', ':position', ':created_at', ':created_meta', ':updated_at', ':updated_meta', ':meta']
trimmed_df = pandas_df.drop(columns=columns_to_drop)
trimmed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307645 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   calendar_year     307645 non-null  object
 1   cal_month_num     307645 non-null  object
 2   supplier          307478 non-null  object
 3   item_code         307645 non-null  object
 4   item_description  307645 non-null  object
 5   item_type         307644 non-null  object
 6   rtl_sales         307642 non-null  object
 7   rtl_transfers     307645 non-null  object
 8   whs_sales         307645 non-null  object
dtypes: object(9)
memory usage: 21.1+ MB


In [6]:
#create a copy of the df
annual_sales_df = trimmed_df.copy()
#rename the columns
annual_sales_df.rename(columns={
    "calendar_year": "year",
    "cal_month_num": "month",
    "rtl_sales": "retail_sales",
    "rtl_transfers": "retail_transfers",
    "whs_sales": "warehouse_sales"
}, inplace=True)
annual_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307645 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   year              307645 non-null  object
 1   month             307645 non-null  object
 2   supplier          307478 non-null  object
 3   item_code         307645 non-null  object
 4   item_description  307645 non-null  object
 5   item_type         307644 non-null  object
 6   retail_sales      307642 non-null  object
 7   retail_transfers  307645 non-null  object
 8   warehouse_sales   307645 non-null  object
dtypes: object(9)
memory usage: 21.1+ MB


In [ ]:
#test = annual_sales_df.copy()
#test['item_code2'] = test['item_code'].apply(lambda x: x if x.isnumeric() else None)
# test[test['item_code2'].notna()]

In [7]:
#Convert 'year'/'month' column to int format (errors!!!!!!!!!!!!!!!!!!!!!)
annual_sales_df['year'] = annual_sales_df['year'].astype('int64')
annual_sales_df['month'] = annual_sales_df['month'].astype('int64')
#change data types of columns 'retail_sales', 'retail_transfers', 'warehouse_sales'
annual_sales_df[['retail_sales', 'retail_transfers', 'warehouse_sales']] = annual_sales_df[['retail_sales', 'retail_transfers', 'warehouse_sales']].astype('float')
#change data types of columns
annual_sales_df['item_code'] = annual_sales_df['item_code'].apply(lambda x: x if x.isnumeric() else None)
annual_sales_df = annual_sales_df[annual_sales_df['item_code'].notna()]
annual_sales_df['item_code'] = annual_sales_df['item_code'].astype(int)
annual_sales_df.info()
#reorganize based on year and month
sorted_annual_df = annual_sales_df.sort_values(by=['year', 'month'], ascending=[False, False])
sorted_annual_df

<class 'pandas.core.frame.DataFrame'>
Index: 307591 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   year              307591 non-null  int64  
 1   month             307591 non-null  int64  
 2   supplier          307471 non-null  object 
 3   item_code         307591 non-null  int32  
 4   item_description  307591 non-null  object 
 5   item_type         307590 non-null  object 
 6   retail_sales      307588 non-null  float64
 7   retail_transfers  307591 non-null  float64
 8   warehouse_sales   307591 non-null  float64
dtypes: float64(3), int32(1), int64(2), object(3)
memory usage: 22.3+ MB


,year,month,supplier,item_code,item_description,item_type,retail_sales,retail_transfers,warehouse_sales
23250,2020,9,LEGENDS LTD,11203,BENEDIKTINER HELLES LAGER - 50L,KEGS,0.00,0.0,1.00
23256,2020,9,Default,104,FOUR BOTTLE WINE TOTE,STR_SUPPLIES,0.20,0.0,0.00
23257,2020,9,REPUBLIC NATIONAL DISTRIBUTING CO,110132,R MONDAVI PRIVATE SEL CAB - 375ML,WINE,1.47,1.0,6.00
23258,2020,9,LEGENDS LTD,11477,REISSDORF KOELSCH - 16.9 OZ 6/4 CAN,BEER,0.00,0.0,3.00
23259,2020,9,PERNOD RICARD USA LLC,11482,MONKEY 47 - 375ML,LIQUOR,0.85,1.0,0.00
...,...,...,...,...,...,...,...,...,...
48511,2017,6,LEGENDS LTD,99753,DUTCHESS DE BOURGOGNE NR - 750ML,BEER,0.16,1.0,6.00
48512,2017,6,ANHEUSER BUSCH INC,9997,HOEGAARDEN 4/6NR - 12OZ,BEER,118.25,116.0,273.75
48513,2017,6,COASTAL BREWING COMPANY LLC,99970,DOMINION OAK BARREL STOUT 4/6 NR - 12OZ,BEER,12.75,9.0,52.00
48514,2017,6,BOSTON BEER CORPORATION,99988,SAM ADAMS HOPSCAPE 1/6 KG,KEGS,0.00,0.0,-1.00


In [8]:
#reset df index
sorted_annual_df.reset_index(drop=True, inplace=True)
sorted_annual_df

,year,month,supplier,item_code,item_description,item_type,retail_sales,retail_transfers,warehouse_sales
0,2020,9,LEGENDS LTD,11203,BENEDIKTINER HELLES LAGER - 50L,KEGS,0.00,0.0,1.00
1,2020,9,Default,104,FOUR BOTTLE WINE TOTE,STR_SUPPLIES,0.20,0.0,0.00
2,2020,9,REPUBLIC NATIONAL DISTRIBUTING CO,110132,R MONDAVI PRIVATE SEL CAB - 375ML,WINE,1.47,1.0,6.00
3,2020,9,LEGENDS LTD,11477,REISSDORF KOELSCH - 16.9 OZ 6/4 CAN,BEER,0.00,0.0,3.00
4,2020,9,PERNOD RICARD USA LLC,11482,MONKEY 47 - 375ML,LIQUOR,0.85,1.0,0.00
...,...,...,...,...,...,...,...,...,...
307586,2017,6,LEGENDS LTD,99753,DUTCHESS DE BOURGOGNE NR - 750ML,BEER,0.16,1.0,6.00
307587,2017,6,ANHEUSER BUSCH INC,9997,HOEGAARDEN 4/6NR - 12OZ,BEER,118.25,116.0,273.75
307588,2017,6,COASTAL BREWING COMPANY LLC,99970,DOMINION OAK BARREL STOUT 4/6 NR - 12OZ,BEER,12.75,9.0,52.00
307589,2017,6,BOSTON BEER CORPORATION,99988,SAM ADAMS HOPSCAPE 1/6 KG,KEGS,0.00,0.0,-1.00


In [29]:
#create a supplier df
#find unique suppliers
unique_suppliers = sorted_annual_df["supplier"].unique()
#create array to generate supplier id
supplier_id = np.arange(1, 398)
#supplier_id = ['supplier#' + str(x) for x in supplier_array]
supplier_df = pd.DataFrame({
    "supplier": unique_suppliers,
    "supplier_id": supplier_id
})
# Export supplier_df as CSV files.
csv_file_path = os.path.join(script_dir, 'suppliers.csv')
supplier_df.to_csv(csv_file_path, index=False)
#supplier_df.reset_index(drop=True, inplace=True)

In [51]:
#create the item df
#columns_to_select = ['item_code', 'item_description']
unique_items = sorted_annual_df[['item_code', 'item_description']].drop_duplicates()
unique_items_df = pd.DataFrame(unique_items)
unique_items_df.reset_index(drop=True, inplace=True)
#export the new df
csv_file_path = os.path.join(script_dir, 'items.csv')
unique_items_df.to_csv(csv_file_path, index=False)


KeyError: "None of [Index(['item_code', 'item_description'], dtype='object')] are in the [columns]"

In [33]:
#create the item_type df
#columns_to_select = ['item_type']
item_types = sorted_annual_df['item_type'].drop_duplicates()
item_types_df = pd.DataFrame(item_types)
# Create a new 'id' column with unique identifiers
item_types_df['type_id'] = item_types_df.reset_index().index + 1
#export the new df
csv_file_path = os.path.join(script_dir, 'item_types.csv')
item_types_df.to_csv(csv_file_path, index=False)

In [34]:
#add a sale_id, supplier_id and item_type_id column (merge with the previously created columns)
sorted_annual_df = sorted_annual_df.merge(item_types_df, on='item_type')
sorted_annual_df = sorted_annual_df.merge(unique_items_df, on='item_code')
sorted_annual_df = sorted_annual_df.merge(supplier_df, on='supplier')
#add the SaleID column
sorted_annual_df['sale_id'] = sorted_annual_df.index

,year,month,supplier,item_code,item_description_x,item_type,retail_sales,retail_transfers,warehouse_sales,type_id,item_description_y,supplier_id,sale_id
0,2020,9,LEGENDS LTD,11203,BENEDIKTINER HELLES LAGER - 50L,KEGS,0.0,0.0,1.0,1,BENEDIKTINER HELLES LAGER - 50L,1,0
1,2020,9,LEGENDS LTD,63600,MAINE LUNCH 1/2 KEG,KEGS,0.0,0.0,4.0,1,MAINE LUNCH 1/2 KEG,1,1
2,2020,7,LEGENDS LTD,63600,MAINE LUNCH 1/2 KEG,KEGS,0.0,0.0,0.0,1,MAINE LUNCH 1/2 KEG,1,2
3,2020,3,LEGENDS LTD,63600,MAINE LUNCH 1/2 KEG,KEGS,0.0,0.0,4.0,1,MAINE LUNCH 1/2 KEG,1,3
4,2020,1,LEGENDS LTD,63600,MAINE LUNCH 1/2 KEG,KEGS,0.0,0.0,6.0,1,MAINE LUNCH 1/2 KEG,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324775,2019,5,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4155.0,8,EMPTY 1/2 KEG (30.00),145,324775
324776,2019,4,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-4299.0,8,EMPTY 1/2 KEG (30.00),145,324776
324777,2019,3,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-3983.0,8,EMPTY 1/2 KEG (30.00),145,324777
324778,2019,2,PREMIUM DISTRIBUTORS INC,175,EMPTY 1/2 KEG (30.00),DUNNAGE,0.0,0.0,-3370.0,8,EMPTY 1/2 KEG (30.00),145,324778


In [41]:
sorted_annual_df.head()

,year,month,item_code,retail_sales,retail_transfers,warehouse_sales,type_id,supplier_id,sale_id
0,2020,9,11203,0.0,0.0,1.0,1,1,0
1,2020,9,63600,0.0,0.0,4.0,1,1,1
2,2020,7,63600,0.0,0.0,0.0,1,1,2
3,2020,3,63600,0.0,0.0,4.0,1,1,3
4,2020,1,63600,0.0,0.0,6.0,1,1,4


In [49]:
#drop unwanted columns
sorted_annual_df.drop(['supplier', 'item_description_x', 'item_type', 'item_description_y'], axis=1, inplace=True)
sorted_annual_df.rename(columns={"item_code": "item_id"}, inplace=True)
#reorder the columns
refined_df = sorted_annual_df[['sale_id', 'year', 'month', 'supplier_id', 'item_id', 'type_id', 'retail_sales', 'retail_transfers', 'warehouse_sales']]
#Specify the file path for the new CSV file
csv_file_path = os.path.join(script_dir, 'refined_df.csv')
#Export the DataFrame to the new CSV file
refined_df.to_csv(csv_file_path, index=False)

In [52]:
refined_df

,sale_id,year,month,supplier_id,item_id,type_id,retail_sales,retail_transfers,warehouse_sales
0,0,2020,9,1,11203,1,0.0,0.0,1.0
1,1,2020,9,1,63600,1,0.0,0.0,4.0
2,2,2020,7,1,63600,1,0.0,0.0,0.0
3,3,2020,3,1,63600,1,0.0,0.0,4.0
4,4,2020,1,1,63600,1,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...
324775,324775,2019,5,145,175,8,0.0,0.0,-4155.0
324776,324776,2019,4,145,175,8,0.0,0.0,-4299.0
324777,324777,2019,3,145,175,8,0.0,0.0,-3983.0
324778,324778,2019,2,145,175,8,0.0,0.0,-3370.0
